### HDL 

#### create dl table by create_table

In [ ]:
from hana_ml.dataframe import ConnectionContext

conn = ConnectionContext(address="f98ad2fb-a57e-4bbb-8509-f31b59074da6.hana.canary-eu10.hanacloud.ondemand.com",
                         port=443,
                         user="MLLAB_DL1",
                         encrypt='true',
                         sslValidateCertificate='false')

In [ ]:
conn.drop_table("DL_TABLE", data_lake=True)
conn.create_table(table="DL_TABLE", table_structure={"A":"INT"}, data_lake=True)

#### create HANA virtual table

In [ ]:
conn.drop_table("DL_TABLE")
conn.create_virtual_table(table="DL_TABLE", data_lake_table="DL_TABLE")

#### query virtual table

In [ ]:
conn.sql("SELECT * FROM DL_TABLE").collect()

#### drop DL table & HANA virtual table

In [ ]:
conn.drop_table("DL_TABLE", data_lake=True)
conn.drop_table("DL_TABLE")

#### copy dataframe to data lake

In [ ]:
import pandas as pd
from hana_ml.dataframe import create_dataframe_from_pandas

df = create_dataframe_from_pandas(conn, pandas_df=pd.DataFrame({"A": [1,2,3]}), table_name="MY_TABLE", force=True)

In [ ]:
df.collect()

In [ ]:
df.save("DL_TABLE", data_lake=True)

In [ ]:
conn.sql("SELECT * FROM DL_TABLE").collect()

In [ ]:
conn.drop_table("DL_TABLE", data_lake=True)
conn.drop_table("DL_TABLE")

In [ ]:
conn.copy_to_data_lake(df, virtual_table="DL_TABLE", data_lake_table="DL_TABLE", append=False)

In [ ]:
conn.sql("SELECT * FROM DL_TABLE").collect()

#### Model Storage in DL

In [ ]:
from data_load_utils import DataSets, Settings
from hana_ml.algorithms.pal.model_selection import GridSearchCV
from hana_ml.algorithms.pal.unified_classification import UnifiedClassification
full_tbl, train_tbl, test_tbl, _ = DataSets.load_diabetes_data(conn)
diabetes_train = conn.table(train_tbl)
diabetes_test = conn.table(test_tbl)

uc_hgbdt = UnifiedClassification('HybridGradientBoostingTree')

gscv = GridSearchCV(estimator=uc_hgbdt, 
                    param_grid={'learning_rate': [0.1, 0.4, 0.7, 1],
                                'n_estimators': [4, 6, 8, 10],
                                'split_threshold': [0.1, 0.4, 0.7, 1]},
                    train_control=dict(fold_num=5,
                                       resampling_method='cv',
                                       random_state=1,
                                       ref_metric=['auc']),
                    scoring='error_rate')
gscv.fit(data=diabetes_train, key= 'ID',
         label='CLASS',
         partition_method='stratified',
         partition_random_state=1,
         stratified_column='CLASS',
         build_report=True)
features = diabetes_train.columns
features.remove('CLASS')
features.remove('ID')
pred_res = gscv.predict(diabetes_test, key='ID', features=features)

In [ ]:
from hana_ml.model_storage import ModelStorage

model_storage = ModelStorage(connection_context=conn)
gscv.estimator.name = 'HGBT' 
gscv.estimator.version = 1
model_storage.save_model(model=gscv.estimator, storage_type='HDL')

In [ ]:
model_storage.list_models()

In [ ]:
model_storage.delete_model("HGBT", 1)
model_storage.list_models()